In [79]:
## TODO: improve the regex detection and other shit

In [207]:
import re
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [208]:
df = pd.read_csv('processed.csv')

In [209]:
df.head()

,Unnamed: 0,entity_name,entity_value,image_data,group_id
0,0,weight,500.0 gram,[],748919
1,1,volume,1.0 cup,[],916768
2,2,weight,0.709 gram,"[{'word': '02g', 'metadata': {'word': '02g', '...",459516
3,3,weight,0.709 gram,"[{'word': '2g', 'metadata': {'word': '2g', 'po...",459516
4,4,weight,1400 milligram,[],731432


In [210]:
# Unit normalization mapping
entity_unit_map = {
    'weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

def normalize_unit(word):
    # Convert units to a standard form
    unit_patterns = {
        r'g|gram|grams': 'gram',
        r'kg|kilogram|kilograms': 'kilogram',
        r'cm|centimetre|centimetres': 'centimetre',
        r'mm|millimetre|millimetres': 'millimetre',
        r'inch|inches': 'inch',
        r'foot|feet|ft': 'foot',
        r'lb|lbs|pound|pounds': 'pound'
        # Add more mappings if necessary
    }
    for pattern, unit in unit_patterns.items():
        if re.search(pattern, word, re.IGNORECASE):
            return unit
    return word

In [211]:
def extract_number_and_unit(word):
    patterns = {
        'weight': r'(\d+(\.\d+)?)\s?(g|gram|grams|kg|kilogram|kilograms|lb|lbs|pound|pounds)',
        'length': r'(\d+(\.\d+)?)\s?(cm|centimetre|centimetres|mm|millimetre|millimetres|inch|feet|ft)',
        'voltage': r'(\d+(\.\d+)?)\s?(V|volts|kV|kilovolts)',
        'wattage': r'(\d+(\.\d+)?)\s?(W|watts|kW|kilowatts)',
        'volume': r'(\d+(\.\d+)?)\s?(L|litres|ml|millilitres|gallon|cup|pint|quart)'
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, word, re.IGNORECASE)
        if match:
            value = float(match.group(1))
            unit = normalize_unit(match.group(3))
            return value, unit
    return None, None

In [212]:
import ast

# Function to parse the string and extract features
def extract_data_from_image(image_data_str):
    # Parse the string to convert it to a Python list of dictionaries
    try:
        image_data = ast.literal_eval(image_data_str)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing image_data: {e}")
        return []

    # Ensure image_data is a list
    if isinstance(image_data, list):
        features = []
        for entry in image_data:
            # Each entry should be a dictionary, access 'metadata' and 'position' fields
            metadata = entry.get('metadata', {})
            position = metadata.get('position', {})

            # Extract the relevant fields with default values in case of missing data
            x = position.get('x', 0)
            y = position.get('y', 0)
            width = position.get('width', 0)
            height = position.get('height', 0)
            confidence = metadata.get('confidence', 0)
            word = entry.get('word', '')
            orientation = entry.get('orientation', '0 degrees')

            # Append the extracted features as a dictionary
            features.append({
                'x': x,
                'y': y,
                'width': width,
                'height': height,
                'confidence': confidence,
                'word': word,
                'orientation': orientation
            })

        return features
    else:
        # If image_data is not a list, return an empty list or raise an error
        return []

In [213]:
df['extracted_image_data'] = df['image_data'].apply(extract_data_from_image)

In [214]:
df = df.drop(columns=['Unnamed: 0'])

In [215]:
df.head()

,entity_name,entity_value,image_data,group_id,extracted_image_data
0,weight,500.0 gram,[],748919,[]
1,volume,1.0 cup,[],916768,[]
2,weight,0.709 gram,"[{'word': '02g', 'metadata': {'word': '02g', '...",459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2..."
3,weight,0.709 gram,"[{'word': '2g', 'metadata': {'word': '2g', 'po...",459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1..."
4,weight,1400 milligram,[],731432,[]


In [216]:
df = df.drop(columns = ['image_data'])

In [217]:
df.head()

,entity_name,entity_value,group_id,extracted_image_data
0,weight,500.0 gram,748919,[]
1,volume,1.0 cup,916768,[]
2,weight,0.709 gram,459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2..."
3,weight,0.709 gram,459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1..."
4,weight,1400 milligram,731432,[]


In [218]:
## extraction of actual text

unit_conversion = {
    'g': 1, 'gram': 1, 'grams': 1,
    'kg': 1000, 'kilogram': 1000, 'kilograms': 1000,
    'mg': 0.001, 'milligram': 0.001,
    'cm': 1, 'centimeter': 1, 'centimeters': 1,
    'm': 100, 'meter': 100, 'meters': 100,
    'inch': 2.54, 'foot': 30.48
}

def convert_to_common_unit(value, unit):
    unit = unit.lower()
    if unit in unit_conversion:
        return value * unit_conversion[unit]
    return value

In [219]:
import re

def extract_number_and_unit(text):
    # Regular expression to extract number and unit
    match = re.search(r'(\d+(\.\d+)?)\s*(\w+)', text)
    if match:
        number = float(match.group(1))
        unit = match.group(3).lower()
        return number, unit
    return None, None

def normalize_value(text):
    # Normalize values by extracting number and unit
    number, unit = extract_number_and_unit(text)
    if number is not None and unit is not None:
        return convert_to_common_unit(number, unit)
    return None

In [220]:
def match_entity_value(row):
    # Extract number and unit from entity_value
    entity_value = row['entity_value']
    entity_number, entity_unit = extract_number_and_unit(entity_value)
    
    if entity_number is not None and entity_unit is not None:
        entity_value_normalized = convert_to_common_unit(entity_number, entity_unit)
        
        matched = []
        for item in row['extracted_image_data']:
            word = item['word']
            image_number = normalize_value(word)
            if image_number is not None and image_number == entity_value_normalized:
                matched.append(1)
            else:
                matched.append(0)
        return matched
    else:
        return [0] * len(row['extracted_image_data'])

In [221]:
df['labels'] = df.apply(match_entity_value, axis=1)

In [222]:
df.head()

,entity_name,entity_value,group_id,extracted_image_data,labels
0,weight,500.0 gram,748919,[],[]
1,volume,1.0 cup,916768,[],[]
2,weight,0.709 gram,459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2...","[0, 0, 0, 0, 0]"
3,weight,0.709 gram,459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,weight,1400 milligram,731432,[],[]


In [223]:
# Prepare features like x, y, width, height, orientation, confidence
def extract_features(image_data):
    features = []
    for item in image_data:
        features.append([item['x'], item['y'], item['width'], item['height'], item['orientation'], item['confidence']])
    return np.array(features).flatten()

df['features'] = df['extracted_image_data'].apply(extract_features)

In [224]:
df

,entity_name,entity_value,group_id,extracted_image_data,labels,features
0,weight,500.0 gram,748919,[],[],[]
1,volume,1.0 cup,916768,[],[],[]
2,weight,0.709 gram,459516,"[{'x': 723, 'y': 698, 'width': 34, 'height': 2...","[0, 0, 0, 0, 0]","[723, 698, 34, 22, 0 degrees, 69, 289, 328, 17..."
3,weight,0.709 gram,459516,"[{'x': 457, 'y': 713, 'width': 30, 'height': 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[457, 713, 30, 10, 0 degrees, 12, 422, 827, 8,..."
4,weight,1400 milligram,731432,[],[],[]
5,weight,1400 milligram,731432,[],[],[]
6,weight,1400 milligram,731432,"[{'x': 1031, 'y': 864, 'width': 30, 'height': ...",[0],"[1031, 864, 30, 52, 0 degrees, 57]"
7,weight,1400 milligram,731432,[],[],[]
8,weight,1400 milligram,731432,[],[],[]
9,weight,1400 milligram,731432,[],[],[]


In [225]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [231]:
def prepare_features_and_labels(df):
    expanded_features = []
    expanded_labels = []

    for idx, row in df.iterrows():
        features = row['features']
        labels = row['labels']
        group_id = row['group_id']
        entity_name = row['entity_name']
        
        
        # Split features and labels

        ## TODO : this is bad do not hardcode here, make a global constant
        num_features_per_sample = 6
        num_samples = len(labels)

        for i in range(num_samples):
            feature_set = features[i*num_features_per_sample:(i+1)*num_features_per_sample]
            label = labels[i]
            # risky code
            feature_set = np.append(feature_set,group_id)
            feature_set = np.append(feature_set,entity_name)
            expanded_features.append(feature_set)
            print(feature_set)

            expanded_labels.append(label)
    
    return pd.DataFrame(expanded_features, columns=[f'feature_{i}' for i in range(len(expanded_features[0]))]), np.array(expanded_labels)

# Prepare data
features_df, labels_array = prepare_features_and_labels(df)


['723' '698' '34' '22' '0 degrees' '69' '459516' 'weight']
['289' '328' '17' '16' '0 degrees' '57' '459516' 'weight']
['499' '753' '17' '25' '0 degrees' '0' '459516' 'weight']
['439' '651' '17' '17' '0 degrees' '29' '459516' 'weight']
['361' '723' '22' '34' '0 degrees' '69' '459516' 'weight']
['457' '713' '30' '10' '0 degrees' '12' '459516' 'weight']
['422' '827' '8' '46' '0 degrees' '0' '459516' 'weight']
['348' '858' '9' '20' '0 degrees' '42' '459516' 'weight']
['554' '375' '26' '8' '0 degrees' '37' '459516' 'weight']
['375' '403' '9' '8' '0 degrees' '8' '459516' 'weight']
['690' '623' '18' '8' '0 degrees' '41' '459516' 'weight']
['689' '637' '24' '8' '0 degrees' '35' '459516' 'weight']
['528' '380' '32' '10' '0 degrees' '30' '459516' 'weight']
['535' '598' '27' '10' '0 degrees' '27' '459516' 'weight']
['1031' '864' '30' '52' '0 degrees' '57' '731432' 'weight']
['1386' '43' '187' '63' '0 degrees' '91' '149159' 'weight']
['549' '313' '121' '42' '0 degrees' '92' '149159' 'weight']
['14

In [232]:
features_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7
0,723,698,34,22,0 degrees,69,459516,weight
1,289,328,17,16,0 degrees,57,459516,weight
2,499,753,17,25,0 degrees,0,459516,weight
3,439,651,17,17,0 degrees,29,459516,weight
4,361,723,22,34,0 degrees,69,459516,weight
5,457,713,30,10,0 degrees,12,459516,weight
6,422,827,8,46,0 degrees,0,459516,weight
7,348,858,9,20,0 degrees,42,459516,weight
8,554,375,26,8,0 degrees,37,459516,weight
9,375,403,9,8,0 degrees,8,459516,weight


In [235]:
final_features = features_df
final_features

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7
0,723,698,34,22,0 degrees,69,459516,weight
1,289,328,17,16,0 degrees,57,459516,weight
2,499,753,17,25,0 degrees,0,459516,weight
3,439,651,17,17,0 degrees,29,459516,weight
4,361,723,22,34,0 degrees,69,459516,weight
5,457,713,30,10,0 degrees,12,459516,weight
6,422,827,8,46,0 degrees,0,459516,weight
7,348,858,9,20,0 degrees,42,459516,weight
8,554,375,26,8,0 degrees,37,459516,weight
9,375,403,9,8,0 degrees,8,459516,weight


In [236]:
final_labels = pd.DataFrame(labels_array)

In [245]:
final_labels

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [250]:
labelled_data = pd.concat([final_features, final_labels], axis=1)

In [251]:
labelled_data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,0
0,723,698,34,22,0 degrees,69,459516,weight,0
1,289,328,17,16,0 degrees,57,459516,weight,0
2,499,753,17,25,0 degrees,0,459516,weight,0
3,439,651,17,17,0 degrees,29,459516,weight,0
4,361,723,22,34,0 degrees,69,459516,weight,0
5,457,713,30,10,0 degrees,12,459516,weight,0
6,422,827,8,46,0 degrees,0,459516,weight,0
7,348,858,9,20,0 degrees,42,459516,weight,0
8,554,375,26,8,0 degrees,37,459516,weight,0
9,375,403,9,8,0 degrees,8,459516,weight,0


In [ ]:
labelled_data.to_csv('labelled_data.csv')